# OpenSearch Neural Sparse Integration Test

This notebook tests the korean-neural-sparse-encoder-v1 model with OpenSearch.

**Approach**: External embedding - Generate sparse vectors locally and index to OpenSearch.

## Prerequisites

- AWS credentials configured (default profile)
- OpenSearch domain: ltr-vector.awsbuddy.com
- Required packages: opensearch-py, requests-aws4auth, boto3, transformers, torch

In [8]:
import sys
from pathlib import Path

# Add project root to path
PROJECT_ROOT = Path.cwd().parent
HUGGINGFACE_DIR = PROJECT_ROOT / "huggingface"
sys.path.insert(0, str(HUGGINGFACE_DIR))

print(f"Project root: {PROJECT_ROOT}")
print(f"Huggingface dir: {HUGGINGFACE_DIR}")

Project root: /home/west/Documents/cursor-workspace/opensearch-neural-pre-train
Huggingface dir: /home/west/Documents/cursor-workspace/opensearch-neural-pre-train/huggingface


In [9]:
import json
import time
from typing import Dict, List, Any

import boto3
import torch
from opensearchpy import OpenSearch, RequestsHttpConnection
from opensearchpy.helpers import bulk
from requests_aws4auth import AWS4Auth
from transformers import AutoTokenizer, AutoConfig
from safetensors.torch import load_file

from modeling_splade import SPLADEModel

print("Packages imported successfully")

Packages imported successfully


## 1. Load Neural Sparse Model (Local)

In [10]:
# Load model from local huggingface folder
tokenizer = AutoTokenizer.from_pretrained(str(HUGGINGFACE_DIR))
config = AutoConfig.from_pretrained(str(HUGGINGFACE_DIR))
model = SPLADEModel(config)

# Load weights
state_dict = load_file(str(HUGGINGFACE_DIR / "model.safetensors"))
model.load_state_dict(state_dict, strict=False)
model.eval()

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(f"Model loaded on {device}")
print(f"Vocab size: {config.vocab_size}")

/home/west/Documents/cursor-workspace/opensearch-neural-pre-train/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:435: UserWarning: 
    Found GPU0 NVIDIA GB10 which is of cuda capability 12.1.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (8.0) - (12.0)
    
  queued_call()


Model loaded on cuda
Vocab size: 50000


In [11]:
def encode_sparse(text: str, top_k: int = 100) -> Dict[str, float]:
    """
    Encode text to sparse vector.
    
    Returns dict of {token: weight} for non-zero weights.
    """
    inputs = tokenizer(
        text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=64,
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        sparse_repr, _ = model(**inputs)
    
    # Get top-k non-zero values
    weights = sparse_repr[0].cpu()
    top_values, top_indices = weights.topk(top_k)
    
    result = {}
    for idx, val in zip(top_indices.tolist(), top_values.tolist()):
        if val > 0:
            token = tokenizer.decode([idx]).strip()
            if token and not token.startswith("##"):  # Skip subword tokens for cleaner output
                result[token] = round(val, 4)
    
    return result


def encode_sparse_full(text: str, min_weight: float = 0.1) -> Dict[str, float]:
    """
    Encode text to sparse vector with all tokens above threshold.
    
    Returns dict of {token_id: weight} for weights > min_weight.
    Uses token IDs as keys for OpenSearch compatibility.
    """
    inputs = tokenizer(
        text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=64,
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        sparse_repr, _ = model(**inputs)
    
    weights = sparse_repr[0].cpu()
    
    # Get all non-zero values above threshold
    non_zero_mask = weights > min_weight
    non_zero_indices = non_zero_mask.nonzero(as_tuple=True)[0]
    
    result = {}
    for idx in non_zero_indices.tolist():
        val = weights[idx].item()
        token = tokenizer.decode([idx]).strip()
        if token:  # Skip empty tokens
            # Use token string as key (OpenSearch rank_features requires string keys)
            result[token] = round(val, 4)
    
    return result

In [12]:
# Test encoding
test_queries = ["손해배상", "인공지능", "진단"]

for query in test_queries:
    sparse = encode_sparse(query, top_k=10)
    print(f"\n'{query}':")
    for token, weight in list(sparse.items())[:10]:
        print(f"  {token}: {weight}")

W1228 15:21:00.375000 3292845 torch/_inductor/utils.py:1661] [1/0_1] Not enough SMs to use max_autotune_gemm mode



'손해배상':
  손해: 3.1094
  상: 2.6562
  배: 2.3906
  보상: 2.2812
  피해: 2.2656
  손실: 2.2344
  손: 2.2031

'인공지능':
  인공지능: 3.1094
  AI: 2.5781
  지능: 2.4062
  알고리즘: 2.3125
  로봇: 2.1406
  인공: 2.0938
  컴퓨팅: 2.0625
  플랫폼: 2.0469
  컴퓨터: 2.0469

'진단':
  진단: 3.2188
  검진: 2.5469
  diagn: 2.3906
  검사: 2.3906
  확진: 2.3438
  진료: 2.3125
  점검: 2.2656
  측정: 2.2656


## 2. Connect to OpenSearch

In [13]:
# OpenSearch configuration
OPENSEARCH_HOST = "ltr-vector.awsbuddy.com"
OPENSEARCH_PORT = 443
AWS_REGION = "us-east-1"

# Get AWS credentials
credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(
    credentials.access_key,
    credentials.secret_key,
    AWS_REGION,
    "es",
    session_token=credentials.token,
)

# Create OpenSearch client
client = OpenSearch(
    hosts=[{"host": OPENSEARCH_HOST, "port": OPENSEARCH_PORT}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=60,
)

# Test connection
info = client.info()
print(f"Connected to OpenSearch")
print(f"Cluster name: {info['cluster_name']}")

Connected to OpenSearch
Cluster name: 505725882051:ltr-vector


## 3. Create Index with rank_features Field

In [14]:
INDEX_NAME = "korean-neural-sparse-test"

# Delete index if exists
try:
    client.indices.delete(index=INDEX_NAME)
    print(f"Deleted existing index: {INDEX_NAME}")
except Exception:
    pass

# Create index with rank_features for sparse vectors
index_body = {
    "settings": {
        "index": {
            "number_of_shards": 1,
            "number_of_replicas": 0,
        }
    },
    "mappings": {
        "properties": {
            "title": {"type": "text", "analyzer": "standard"},
            "content": {"type": "text", "analyzer": "standard"},
            "category": {"type": "keyword"},
            "title_sparse": {"type": "rank_features"},
            "content_sparse": {"type": "rank_features"},
        }
    },
}

try:
    response = client.indices.create(index=INDEX_NAME, body=index_body)
    print(f"Index created: {INDEX_NAME}")
except Exception as e:
    print(f"Error creating index: {e}")

Index created: korean-neural-sparse-test


## 4. Prepare and Index Documents

In [15]:
# Korean test documents (legal, medical, tech, business, general)
DOCUMENTS = [
    # Legal domain
    {
        "title": "손해배상 청구 소송",
        "content": "피고는 원고에게 발생한 손해에 대하여 배상할 책임이 있습니다. 본 사건에서 피고의 과실로 인해 원고가 입은 손해를 산정합니다.",
        "category": "legal",
    },
    {
        "title": "계약 위반 및 해지",
        "content": "계약 당사자 일방이 계약상 의무를 위반한 경우, 상대방은 계약을 해지할 수 있습니다. 계약 해지 시 원상회복 의무가 발생합니다.",
        "category": "legal",
    },
    {
        "title": "부동산 매매 계약서",
        "content": "매도인과 매수인은 아래 부동산에 대하여 매매계약을 체결합니다. 매매대금은 계약금, 중도금, 잔금으로 나누어 지급합니다.",
        "category": "legal",
    },
    {
        "title": "형사 소송 절차 안내",
        "content": "형사 소송은 수사, 기소, 공판, 판결의 순서로 진행됩니다. 피고인은 변호인의 조력을 받을 권리가 있습니다.",
        "category": "legal",
    },
    {
        "title": "이혼 소송과 재산분할",
        "content": "이혼 시 부부가 혼인 중 공동으로 형성한 재산은 분할의 대상이 됩니다. 재산분할 비율은 기여도에 따라 결정됩니다.",
        "category": "legal",
    },
    # Medical domain
    {
        "title": "당뇨병 진단 및 치료",
        "content": "당뇨병은 혈당 수치가 비정상적으로 높은 상태입니다. 진단을 위해 공복혈당 검사와 당화혈색소 검사를 시행합니다.",
        "category": "medical",
    },
    {
        "title": "고혈압 관리 가이드",
        "content": "고혈압은 혈압이 정상 범위를 초과하는 상태입니다. 생활습관 개선과 약물 치료를 통해 혈압을 조절할 수 있습니다.",
        "category": "medical",
    },
    {
        "title": "암 조기 검진의 중요성",
        "content": "암은 조기에 발견하면 완치율이 높습니다. 정기적인 건강검진을 통해 암을 조기에 발견할 수 있습니다.",
        "category": "medical",
    },
    {
        "title": "감기와 독감의 차이",
        "content": "감기와 독감은 모두 호흡기 질환이지만 원인 바이러스가 다릅니다. 독감은 고열과 근육통이 심하게 나타납니다.",
        "category": "medical",
    },
    {
        "title": "우울증 치료 방법",
        "content": "우울증은 전문적인 치료가 필요한 정신건강 질환입니다. 약물치료와 심리상담을 병행하면 효과적입니다.",
        "category": "medical",
    },
    # IT/Tech domain
    {
        "title": "인공지능 기술 동향",
        "content": "인공지능과 머신러닝 기술이 빠르게 발전하고 있습니다. 딥러닝 모델은 자연어 처리와 이미지 인식에서 뛰어난 성능을 보입니다.",
        "category": "tech",
    },
    {
        "title": "클라우드 컴퓨팅 서비스",
        "content": "클라우드 컴퓨팅은 인터넷을 통해 컴퓨팅 자원을 제공하는 서비스입니다. AWS, Azure, GCP가 대표적인 클라우드 서비스입니다.",
        "category": "tech",
    },
    {
        "title": "검색 엔진 최적화 가이드",
        "content": "검색 엔진 최적화(SEO)는 웹사이트가 검색 결과에서 상위에 노출되도록 하는 기법입니다. 키워드 분석과 콘텐츠 최적화가 중요합니다.",
        "category": "tech",
    },
    {
        "title": "데이터베이스 설계 원칙",
        "content": "효율적인 데이터베이스 설계는 데이터 중복을 최소화하고 무결성을 보장합니다. 정규화를 통해 테이블 구조를 최적화합니다.",
        "category": "tech",
    },
    {
        "title": "사이버 보안 위협 대응",
        "content": "사이버 공격으로부터 시스템을 보호하기 위해 방화벽, 암호화, 접근 제어 등의 보안 조치가 필요합니다.",
        "category": "tech",
    },
    # General/Business domain
    {
        "title": "효과적인 마케팅 전략",
        "content": "고객의 니즈를 파악하고 타겟 시장을 선정하는 것이 마케팅의 기본입니다. 디지털 마케팅과 SNS 활용이 중요해지고 있습니다.",
        "category": "business",
    },
    {
        "title": "재무제표 분석 방법",
        "content": "재무제표는 기업의 재무 상태를 보여주는 문서입니다. 손익계산서, 대차대조표, 현금흐름표를 통해 기업을 분석할 수 있습니다.",
        "category": "business",
    },
    {
        "title": "스타트업 창업 가이드",
        "content": "스타트업 창업 시 사업계획서 작성, 투자 유치, 팀 구성이 중요합니다. MVP를 통해 아이디어를 검증하세요.",
        "category": "business",
    },
    {
        "title": "리더십과 팀 관리",
        "content": "좋은 리더는 팀원들과 소통하고 동기부여를 합니다. 목표 설정과 피드백이 팀 성과 향상에 중요합니다.",
        "category": "business",
    },
    {
        "title": "투자와 자산관리",
        "content": "분산 투자를 통해 위험을 줄이고 장기적인 수익을 추구합니다. 주식, 채권, 부동산 등 다양한 자산에 투자하세요.",
        "category": "business",
    },
    # Additional documents
    {
        "title": "환경 보호와 지속가능성",
        "content": "기후변화에 대응하기 위해 탄소 배출을 줄이고 재생에너지를 활용해야 합니다. 지속가능한 발전이 중요합니다.",
        "category": "general",
    },
    {
        "title": "교육의 미래와 온라인 학습",
        "content": "온라인 교육 플랫폼이 확산되면서 언제 어디서나 학습이 가능해졌습니다. 개인화된 학습 경험을 제공합니다.",
        "category": "general",
    },
    {
        "title": "건강한 식단 관리",
        "content": "균형 잡힌 식단은 건강 유지의 기본입니다. 채소, 과일, 단백질을 적절히 섭취하고 가공식품은 줄이세요.",
        "category": "general",
    },
    {
        "title": "여행 계획 세우기",
        "content": "여행지 선정, 숙소 예약, 일정 계획이 여행 준비의 핵심입니다. 현지 문화와 음식을 미리 조사하세요.",
        "category": "general",
    },
    {
        "title": "스트레스 관리 방법",
        "content": "스트레스는 현대인의 건강을 위협합니다. 운동, 명상, 취미활동을 통해 스트레스를 효과적으로 해소하세요.",
        "category": "general",
    },
]

print(f"Total documents: {len(DOCUMENTS)}")

Total documents: 25


In [16]:
# Generate sparse vectors for all documents
print("Generating sparse vectors...")

for i, doc in enumerate(DOCUMENTS):
    # Generate sparse vectors for title and content
    doc["title_sparse"] = encode_sparse_full(doc["title"], min_weight=0.5)
    doc["content_sparse"] = encode_sparse_full(doc["content"], min_weight=0.5)
    
    if (i + 1) % 5 == 0:
        print(f"  Processed {i + 1}/{len(DOCUMENTS)} documents")

print(f"Done! Generated sparse vectors for {len(DOCUMENTS)} documents")

Generating sparse vectors...
  Processed 5/25 documents
  Processed 10/25 documents
  Processed 15/25 documents
  Processed 20/25 documents
  Processed 25/25 documents
Done! Generated sparse vectors for 25 documents


In [17]:
# Check a sample document
sample_doc = DOCUMENTS[0]
print(f"Sample document:")
print(f"  Title: {sample_doc['title']}")
print(f"  Category: {sample_doc['category']}")
print(f"  Title sparse tokens ({len(sample_doc['title_sparse'])}): {list(sample_doc['title_sparse'].items())[:5]}...")
print(f"  Content sparse tokens ({len(sample_doc['content_sparse'])}): {list(sample_doc['content_sparse'].items())[:5]}...")

Sample document:
  Title: 손해배상 청구 소송
  Category: legal
  Title sparse tokens (3072): [('<unused5>', 0.5469), ('#', 0.9727), ('%', 0.5703), (',', 1.1016), ('-', 0.9883)]...
  Content sparse tokens (28269): [('<unk>', 1.8281), ('<mask>', 0.5625), ('<unused17>', 0.7109), ('<unused21>', 0.5195), ('<unused23>', 0.5703)]...


In [18]:
# Index documents
def index_documents(documents: List[Dict]):
    """Index documents with pre-computed sparse vectors."""
    actions = []
    for i, doc in enumerate(documents):
        action = {
            "_index": INDEX_NAME,
            "_id": str(i + 1),
            "_source": doc,
        }
        actions.append(action)
    
    try:
        success, errors = bulk(client, actions, refresh=True)
        print(f"Indexed {success} documents")
        if errors:
            print(f"Errors: {errors}")
    except Exception as e:
        print(f"Error indexing: {e}")

index_documents(DOCUMENTS)

Indexed 25 documents


In [19]:
# Verify indexed documents
response = client.count(index=INDEX_NAME)
print(f"Document count: {response['count']}")

Document count: 25


## 5. Search with rank_feature Query

In [20]:
def sparse_search(query: str, field: str = "content_sparse", size: int = 5) -> List[Dict]:
    """
    Perform sparse vector search using rank_feature query.
    
    Generates sparse vector for query and searches using rank_feature.
    """
    # Generate query sparse vector
    query_sparse = encode_sparse_full(query, min_weight=0.5)
    
    # Build rank_feature query for each token
    should_clauses = []
    for token, weight in query_sparse.items():
        should_clauses.append({
            "rank_feature": {
                "field": f"{field}.{token}",
                "boost": weight,
            }
        })
    
    if not should_clauses:
        print(f"No valid tokens for query: {query}")
        return []
    
    search_body = {
        "size": size,
        "query": {
            "bool": {
                "should": should_clauses,
            }
        },
        "_source": ["title", "category", "content"],
    }
    
    try:
        response = client.search(index=INDEX_NAME, body=search_body)
        return response["hits"]["hits"]
    except Exception as e:
        print(f"Search error: {e}")
        return []


def print_results(query: str, results: List[Dict]):
    """Print search results."""
    print(f"\nQuery: '{query}'")
    print("-" * 60)
    for i, hit in enumerate(results, 1):
        print(f"{i}. [{hit['_source']['category']}] {hit['_source']['title']}")
        print(f"   Score: {hit['_score']:.4f}")
    if not results:
        print("No results found")

In [21]:
# Test sparse search
TEST_QUERIES = [
    "손해배상",
    "계약 위반",
    "당뇨병 치료",
    "암 검진",
    "인공지능",
    "검색 엔진",
    "투자 전략",
    "스트레스 해소",
]

for query in TEST_QUERIES:
    results = sparse_search(query)
    print_results(query, results)


Query: '손해배상'
------------------------------------------------------------
1. [legal] 손해배상 청구 소송
   Score: 964.5612
2. [legal] 계약 위반 및 해지
   Score: 926.5616
3. [legal] 부동산 매매 계약서
   Score: 894.8417
4. [legal] 이혼 소송과 재산분할
   Score: 861.4494
5. [business] 재무제표 분석 방법
   Score: 845.6030

Query: '계약 위반'
------------------------------------------------------------
1. [legal] 계약 위반 및 해지
   Score: 956.9678
2. [legal] 손해배상 청구 소송
   Score: 889.4637
3. [legal] 부동산 매매 계약서
   Score: 876.0530
4. [legal] 형사 소송 절차 안내
   Score: 848.0267
5. [legal] 이혼 소송과 재산분할
   Score: 842.7261

Query: '당뇨병 치료'
------------------------------------------------------------
1. [medical] 당뇨병 진단 및 치료
   Score: 1539.4568
2. [medical] 고혈압 관리 가이드
   Score: 1492.2563
3. [medical] 감기와 독감의 차이
   Score: 1459.8280
4. [medical] 암 조기 검진의 중요성
   Score: 1401.4888
5. [general] 건강한 식단 관리
   Score: 1400.3647

Query: '암 검진'
------------------------------------------------------------
1. [medical] 암 조기 검진의 중요성
   Score: 756.8114
2. [medica

## 6. Compare with BM25 Search

In [22]:
def bm25_search(query: str, size: int = 5) -> List[Dict]:
    """Perform BM25 text search."""
    search_body = {
        "size": size,
        "query": {
            "multi_match": {
                "query": query,
                "fields": ["title^2", "content"],
            }
        },
        "_source": ["title", "category"],
    }
    
    try:
        response = client.search(index=INDEX_NAME, body=search_body)
        return response["hits"]["hits"]
    except Exception as e:
        print(f"Search error: {e}")
        return []


def compare_search(query: str):
    """Compare sparse vs BM25 search."""
    print(f"\n{'='*70}")
    print(f"Query: '{query}'")
    print("="*70)
    
    # Show query expansion
    query_sparse = encode_sparse(query, top_k=10)
    print(f"\n[Query Expansion]")
    print(f"  {', '.join([f'{t}({w:.2f})' for t, w in list(query_sparse.items())[:8]])}")
    
    print(f"\n[Sparse Search Results]")
    sparse_results = sparse_search(query)
    for i, hit in enumerate(sparse_results[:3], 1):
        print(f"  {i}. [{hit['_source']['category']}] {hit['_source']['title']} (score: {hit['_score']:.2f})")
    
    print(f"\n[BM25 Results]")
    bm25_results = bm25_search(query)
    for i, hit in enumerate(bm25_results[:3], 1):
        print(f"  {i}. [{hit['_source']['category']}] {hit['_source']['title']} (score: {hit['_score']:.2f})")

In [ ]:
# Compare searches - these queries test synonym expansion
COMPARISON_QUERIES = [
    "배상 청구",       # Should find 손해배상 documents
    "병원 진찰",       # Should find 진단/치료 documents  
    "AI 기술",         # Should find 인공지능 documents
    "약정 체결",       # Should find 계약 documents
    "투자 수익",       # Should find 투자/자산관리 documents
]

for query in COMPARISON_QUERIES:
    compare_search(query)


Query: '배상 청구'

[Query Expansion]
  청구(3.16), 배상(3.11), 구(2.50), 보상(2.48), 요구(2.38), 청(2.27)

[Sparse Search Results]
  1. [legal] 계약 위반 및 해지 (score: 1131.26)
  2. [legal] 손해배상 청구 소송 (score: 1113.73)
  3. [legal] 부동산 매매 계약서 (score: 1065.51)

[BM25 Results]
  1. [legal] 손해배상 청구 소송 (score: 2.67)

Query: '병원 진찰'

[Query Expansion]
  병원(3.12), 진(3.08), 병실(2.52), 의료(2.50), 진료(2.45), 진이(2.42), 질(2.41)

[Sparse Search Results]
  1. [medical] 암 조기 검진의 중요성 (score: 957.50)
  2. [legal] 형사 소송 절차 안내 (score: 899.46)
  3. [medical] 당뇨병 진단 및 치료 (score: 895.52)

[BM25 Results]

Query: 'AI 기술'

[Query Expansion]
  기술(3.14), AI(3.11), 인공지능(2.69), 기술력(2.45), IT(2.42), technology(2.42), 기(2.39), Technology(2.38)

[Sparse Search Results]
  1. [tech] 인공지능 기술 동향 (score: 1045.30)
  2. [tech] 검색 엔진 최적화 가이드 (score: 1032.57)
  3. [tech] 클라우드 컴퓨팅 서비스 (score: 1032.43)

[BM25 Results]
  1. [tech] 인공지능 기술 동향 (score: 2.67)

Query: '약정 체결'

[Query Expansion]
  약정(3.03), 체결(2.97), 계약(2.61), 협정(2.42), 협약(2.41), 약속(2.38

: 

## 7. Cleanup (Optional)

In [ ]:
def cleanup():
    """Clean up resources."""
    try:
        client.indices.delete(index=INDEX_NAME)
        print(f"Deleted index: {INDEX_NAME}")
    except Exception as e:
        print(f"Error deleting index: {e}")

# Uncomment to cleanup
# cleanup()

## Summary

This notebook demonstrates:

1. **Local Model Loading**: Load korean-neural-sparse-encoder-v1 from huggingface folder
2. **Sparse Vector Generation**: Generate sparse vectors using the model
3. **OpenSearch Indexing**: Index documents with pre-computed sparse vectors using `rank_features` field
4. **Sparse Search**: Search using `rank_feature` query with query expansion
5. **BM25 Comparison**: Compare sparse search with traditional BM25

### Key Findings

The sparse search should show better synonym matching:
- "배상 청구" → finds "손해배상" documents (synonym expansion)
- "AI 기술" → finds "인공지능" documents (AI ↔ 인공지능)
- "약정 체결" → finds "계약" documents (약정 ↔ 계약)